In [1]:
import openpyxl
import pandas as pd
import os
import tkinter as tk
from tkinter import filedialog
from datetime import datetime

In [2]:
## 파일 경로 받아오는 GUI 호출

root = tk.Tk()
root.withdraw()
dir_path = filedialog.askopenfile(
    parent=root,initialdir=os.getcwd(),title='Please select a file',
    filetypes=(('xls files','*.xls'),('all files','*.*')))

In [9]:
## 파일 읽어서 Data Frame으로 저장

print("load file: ", dir_path.name)
df = pd.read_excel(dir_path.name, sheet_name=None)

mri_table = pd.DataFrame()
mri_table = pd.concat(df, ignore_index=True)

load file:  C:/Users/admin/Desktop/0831.xls
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


In [10]:
# 필요없는 값들 다 지우기

mri_table.drop([0, 1, 2, 3], inplace=True)
mri_table.dropna(axis=1, inplace=True)
mri_table.rename(columns=mri_table.iloc[0],inplace=True)
mri_table.drop([4], inplace=True)

# str --> int 형변환

mri_table["NO"] = pd.to_numeric(mri_table["NO"])
mri_table["등록번호"] = pd.to_numeric(mri_table["등록번호"])

In [11]:
## 나이 필터링 (17세 미만 제외)

pre_mil = mri_table[pd.to_numeric(mri_table["주민번호"].str[7]) < 3]
pre_mil["주민번호"] = "19" + pre_mil["주민번호"]

post_mil = mri_table[pd.to_numeric(mri_table["주민번호"].str[7]) > 2]
post_mil["주민번호"] = "20" + post_mil["주민번호"]

mri_table2 = pd.merge(pre_mil,post_mil, how='outer')
mri_table2 = mri_table2.sort_values(by='NO')
mri_table = mri_table2

mri_table["주민번호"] = mri_table["주민번호"].str[:8]

mri_table["주민번호"] = mri_table["주민번호"].astype('datetime64[ns]')
mri_table = mri_table.rename(columns={'주민번호':'생년월일'})


C:\Users\admin\AppData\Local\Temp/ipykernel_6356/2530370945.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pre_mil["주민번호"] = "19" + pre_mil["주민번호"]
C:\Users\admin\AppData\Local\Temp/ipykernel_6356/2530370945.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  post_mil["주민번호"] = "20" + post_mil["주민번호"]


In [12]:
# 만나이 계산

today = datetime.today()
age = pd.DataFrame()
age["age"] = mri_table['생년월일'].apply(
               lambda x: today.year - x.year - 
               ((today.month, today.day) < (x.month, x.day)) 
               )
mri_table["생년월일"] = age["age"]
mri_table = mri_table.rename(columns={'생년월일':'나이'})

In [13]:
# 나이 17세 미만 환자 삭제
mri_table = mri_table[mri_table["나이"] > 17]

In [14]:
# 필요 없는 열들 삭제
mri_table.drop(["나이","진료일자","병실"],axis=1, inplace=True)

In [17]:
## remove list 파일 읽어서 Data Frame으로 저장

remove_list_path = "./remove_list.xlsx"
df2 = pd.read_excel(remove_list_path, sheet_name=None, engine='openpyxl')

remove_list_table = pd.DataFrame()
remove_list_table = pd.concat(df2, ignore_index=True)

In [18]:
## remove list와 일치하는 처방 삭제하기

for i in range(len(remove_list_table)):
    mri_table = mri_table[~mri_table["처방명"].str.contains(remove_list_table["키워드"][i])]

# No 1부터 다시 매기기
mri_table["NO"] = range(1, len(mri_table)+1)

In [19]:
## data frame을 openpyxl worksheet로 전환

from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows

wb = Workbook()
ws = wb.active

for r in dataframe_to_rows(mri_table, index=False, header=True):
    ws.append(r)

In [20]:
## 서식 넣기

from openpyxl.styles import Alignment, Font, Border, Side
align_center = Alignment(horizontal='center', vertical='center', wrap_text= True)
font_9 = Font(name='굴림', size=9, bold=False)
border_thin = Border(left=Side(style='thin'),right=Side(style='thin'),top=Side(style='thin'),bottom=Side(style='thin'))
col_max = ws.max_column+2

for x in range(1, ws.max_row + 1):
    for y in range(1, col_max):
        ws.cell(row=x, column=y).alignment = align_center
        ws.cell(row=x, column=y).font = font_9
        ws.cell(row=x, column=y).border = border_thin

In [21]:
## 셀 크기 조정
for row in range(1,ws.max_row+1):
    ws.row_dimensions[row].height = 27.75

from openpyxl.utils import get_column_letter
for col in range(1,ws.max_column+2):
    if col == 5:
        ws.column_dimensions[get_column_letter(col)].width = 48
    elif col == 9:
        ws.column_dimensions[get_column_letter(col)].width = 48
    elif col == 10:
        ws.column_dimensions[get_column_letter(col)].width = 32
    else:
        ws.column_dimensions[get_column_letter(col)].width = 13.6

In [22]:
## 최종 결과물을 엑셀 파일로 저장

wb.save("msk_result.xlsx")